In [7]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import googlemaps

gmaps = googlemaps.Client(key='')#here is mine private api

property_df = pd.read_csv('../data/curated/Facilities min-distance.csv')
stations_data = pd.read_csv('../data/raw/ptv_train_station/PTV_METRO_TRAIN_STATION.csv')

properties = property_df[['id', 'addressStreet', 'addressSuburb', 'addressLat', 'addressLng']]
train_stations = stations_data[['STOP_ID', 'STOP_NAME', 'LATITUDE', 'LONGITUDE']]

print(properties.head())
print(stations.head())


         id             addressStreet   addressSuburb  addressLat  addressLng
0  15230553      322/23 Batman Street  WEST MELBOURNE  -37.810738   144.95226
1  15760997     2601/65 Dudley Street  WEST MELBOURNE  -37.810135   144.95040
2  15975970  1212/15-31 Batman Street  WEST MELBOURNE  -37.810757   144.95224
3  16668342      307/33 Batman Street  WEST MELBOURNE  -37.810860   144.95183
4  16803052       2/18 Ireland Street  WEST MELBOURNE  -37.807606   144.94562
   STOP_ID                                          STOP_NAME   LATITUDE  \
0    19970             Royal Park Railway Station (Parkville) -37.781193   
1    19971  Flemington Bridge Railway Station (North Melbo... -37.788140   
2    19972         Macaulay Railway Station (North Melbourne) -37.794267   
3    19973   North Melbourne Railway Station (West Melbourne) -37.807419   
4    19974        Clifton Hill Railway Station (Clifton Hill) -37.788657   

    LONGITUDE  
0  144.952301  
1  144.939323  
2  144.936166  
3  144.9425

In [8]:
properties['min_walking_distance_to_station'] = np.nan
properties['nearest_station'] = None


/tmp/ipykernel_200316/1665391708.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  properties['min_walking_distance_to_station'] = np.nan
/tmp/ipykernel_200316/1665391708.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  properties['nearest_station'] = None


In [10]:
station_coords = train_stations[['LATITUDE', 'LONGITUDE']].to_numpy()
NUM_CLOSEST_STATIONS = 3


In [13]:
for i, property_row in properties.iterrows():
    property_coord = np.array([property_row['addressLat'], property_row['addressLng']])
    distances = np.array([geodesic(property_coord, station_coord).kilometers for station_coord in station_coords])
    closest_indices = np.argsort(distances)[:NUM_CLOSEST_STATIONS]
    nearby_stations = train_stations.iloc[closest_indices]
    
    origins = f"{property_row['addressLat']},{property_row['addressLng']}"
    destinations = [f"{row['LATITUDE']},{row['LONGITUDE']}" for _, row in nearby_stations.iterrows()]
    
    result = gmaps.distance_matrix(origins=origins,
                                   destinations=destinations,
                                   mode="walking")
    
    elements = result['rows'][0]['elements']
    walking_distances = []
    for elem in elements:
        if elem['status'] == 'OK':
            walking_distances.append(elem['distance']['value'])
        else:
            walking_distances.append(np.inf) 
    
    min_index = np.argmin(walking_distances)
    min_distance = walking_distances[min_index] / 1000  
    nearest_station = nearby_stations.iloc[min_index]['STOP_NAME']
    
    properties.at[i, 'min_walking_distance_to_station'] = min_distance
    properties.at[i, 'nearest_station'] = nearest_station


In [14]:
print(properties[['addressStreet', 'addressSuburb', 'min_walking_distance_to_station', 'nearest_station']].head())

              addressStreet   addressSuburb  min_walking_distance_to_station  \
0      322/23 Batman Street  WEST MELBOURNE                            0.485   
1     2601/65 Dudley Street  WEST MELBOURNE                            0.746   
2  1212/15-31 Batman Street  WEST MELBOURNE                            0.487   
3      307/33 Batman Street  WEST MELBOURNE                            0.525   
4       2/18 Ireland Street  WEST MELBOURNE                            0.319   

                                    nearest_station  
0        Flagstaff Railway Station (Melbourne City)  
1        Flagstaff Railway Station (Melbourne City)  
2        Flagstaff Railway Station (Melbourne City)  
3        Flagstaff Railway Station (Melbourne City)  
4  North Melbourne Railway Station (West Melbourne)  


In [17]:
property_data = pd.read_csv('../data/curated/Facilities min-distance.csv')
property_data['min_walking_distance_to_station'] = properties['min_walking_distance_to_station']
property_data['nearest_station'] = properties['nearest_station']
property_data.to_csv('../data/curated/all_Facilities_min_distance.csv', index=False)
print("already")


already
